## 資料讀取

### tar 讀取

In [1]:
import tarfile

In [2]:
# 檔案名稱格式
filename_format="M06A_{year:04d}{month:02d}{day:02d}.tar.gz".format
xz_filename_format="xz/M06A_{year:04d}{month:02d}{day:02d}.tar.xz".format
csv_format = "M06A/{year:04d}{month:02d}{day:02d}/{hour:02d}/TDCS_M06A_{year:04d}{month:02d}{day:02d}_{hour:02d}0000.csv".format

In [3]:
# 打開剛才下載的檔案試試
data_config ={"year":2016, "month":12, "day":18}
tar = tarfile.open(filename_format(**data_config), 'r')

In [4]:
# 如果沒有下載，可以試試看 xz 檔案
#data_dconfig ={"year":2016, "month":11, "day":18}
#tar = tarfile.open(xz_filename_format(**data_config), 'r')

In [5]:
# 列出內容
tar.list()

?rwxrwxrwx nobody/nobody          0 2016-12-19 06:31:22 M06A/20161218/ 
?rwxr-xr-x nobody/nobody          0 2016-12-19 06:31:22 M06A/20161218/19/ 
?rw-r--r-- nobody/nobody   43788302 2016-12-19 06:58:57 M06A/20161218/19/TDCS_M06A_20161218_190000.csv 
?rwxr-xr-x nobody/nobody          0 2016-12-19 06:31:21 M06A/20161218/13/ 
?rw-r--r-- nobody/nobody   47797413 2016-12-19 06:58:52 M06A/20161218/13/TDCS_M06A_20161218_130000.csv 
?rwxr-xr-x nobody/nobody          0 2016-12-19 06:31:22 M06A/20161218/21/ 
?rw-r--r-- nobody/nobody   34892056 2016-12-19 06:58:58 M06A/20161218/21/TDCS_M06A_20161218_210000.csv 
?rwxr-xr-x nobody/nobody          0 2016-12-19 06:31:17 M06A/20161218/00/ 
?rw-r--r-- nobody/nobody   10242555 2016-12-19 06:58:46 M06A/20161218/00/TDCS_M06A_20161218_000000.csv 
?rwxr-xr-x nobody/nobody          0 2016-12-19 06:31:20 M06A/20161218/07/ 
?rw-r--r-- nobody/nobody   29010414 2016-12-19 06:58:48 M06A/20161218/07/TDCS_M06A_20161218_070000.csv 
?rwxr-xr-x nobody/nobody         

In [6]:
# 打開裡面 10 點鐘的資料
csv = tar.extractfile(csv_format(hour=10, **data_config))
# 類似檔案的物件
csv

<ExFileObject name='M06A_20161218.tar.gz'>

In [7]:
# 印出前十行來看看
for i in range(10):
    print(csv.readline().decode())

31,2016-12-18 10:00:50,01F1045N,2016-12-18 10:29:01,01F0584N,53.400,Y,2016-12-18 10:00:50+01F1045N; 2016-12-18 10:04:37+01F0979N; 2016-12-18 10:05:58+01F0956N; 2016-12-18 10:07:43+01F0928N; 2016-12-18 10:10:43+01F0880N; 2016-12-18 10:18:38+01F0750N; 2016-12-18 10:27:26+01H0608N; 2016-12-18 10:29:01+01F0584N

31,2016-12-18 10:34:24,01F3525S,2016-12-18 10:46:09,01F3686S,20.200,Y,2016-12-18 10:34:24+01F3525S; 2016-12-18 10:37:13+01F3561S; 2016-12-18 10:39:21+01F3590S; 2016-12-18 10:42:54+01F3640S; 2016-12-18 10:45:26+01F3676S; 2016-12-18 10:46:09+01F3686S

31,2016-12-18 10:57:33,03F3854N,2016-12-18 11:15:23,01F3686S,33.600,Y,2016-12-18 10:57:33+03F3854N; 2016-12-18 11:12:29+01F3640S; 2016-12-18 11:14:45+01F3676S; 2016-12-18 11:15:23+01F3686S

31,2016-12-18 10:05:11,03F0525S,2016-12-18 10:07:39,03F0559S,12.100,Y,2016-12-18 10:05:11+03F0525S; 2016-12-18 10:07:39+03F0559S

31,2016-12-18 10:35:46,01F3185S,2016-12-18 11:05:08,01F3686S,54.100,Y,2016-12-18 10:35:46+01F3185S; 2016-12-18 10:38:12+

## 利用 pandas 來處理資料

In [8]:
import pandas

csv 欄位依照手冊設定

<a href="http://www.freeway.gov.tw/UserFiles/File/TIMCCC/TDCS%E4%BD%BF%E7%94%A8%E6%89%8B%E5%86%8A(tanfb)v3.0-1.pdf">國道高速公路電子收費交通資料蒐集支援系統(Traffic Data Collection System,TDCS)使用手冊</a>


* VehicleType： 車種 31(小客車) 、32(小貨車) 、41(大客車) 、42(大貨車) 、5(聯結車)
* DetectionTime_O：車輛通過本旅次第 1 個偵測站時間
* GantryID_O： 車輛通過本旅次第 1 個偵測站編號
* DetectionTime_D： 車輛通過本旅次最後 1 個偵測站時間
* GantryID_D： 車輛通過本旅次最後 1 個偵測站編號
* TripLength：本旅次之行駛距離
* TripEnd：旅次結標記 (Y：旅次正常結束，N：異常)
* TripInformation： 本旅次經過的各個偵測站之通過時間及偵測站編號

In [9]:
# 設定欄位名稱
M06A_fields = ['VehicleType',
               'DetectionTime_O','GantryID_O',
               'DetectionTime_D','GantryID_D ',
               'TripLength', 'TripEnd', 'TripInformation']
# 打開裡面 10 點鐘的資料
csv = tar.extractfile(csv_format(hour=10, **data_config))
# 讀進資料
data = pandas.read_csv(csv, names=M06A_fields)

In [10]:
data

,VehicleType,DetectionTime_O,GantryID_O,DetectionTime_D,GantryID_D,TripLength,TripEnd,TripInformation
0,31,2016-12-18 10:00:50,01F1045N,2016-12-18 10:29:01,01F0584N,53.400,Y,2016-12-18 10:00:50+01F1045N; 2016-12-18 10:04...
1,31,2016-12-18 10:34:24,01F3525S,2016-12-18 10:46:09,01F3686S,20.200,Y,2016-12-18 10:34:24+01F3525S; 2016-12-18 10:37...
2,31,2016-12-18 10:57:33,03F3854N,2016-12-18 11:15:23,01F3686S,33.600,Y,2016-12-18 10:57:33+03F3854N; 2016-12-18 11:12...
3,31,2016-12-18 10:05:11,03F0525S,2016-12-18 10:07:39,03F0559S,12.100,Y,2016-12-18 10:05:11+03F0525S; 2016-12-18 10:07...
4,31,2016-12-18 10:35:46,01F3185S,2016-12-18 11:05:08,01F3686S,54.100,Y,2016-12-18 10:35:46+01F3185S; 2016-12-18 10:38...
5,31,2016-12-18 10:00:35,03F3496N,2016-12-18 10:03:32,03F3445N,16.800,Y,2016-12-18 10:00:35+03F3496N; 2016-12-18 10:03...
6,32,2016-12-18 10:00:21,05F0287N,2016-12-18 12:40:50,03F0021N,46.600,Y,2016-12-18 10:00:21+05F0287N; 2016-12-18 12:27...
7,31,2016-12-18 10:50:12,01H0271N,2016-12-18 10:55:02,01H0200N,13.300,Y,2016-12-18 10:50:12+01H0271N; 2016-12-18 10:54...
8,31,2016-12-18 10:09:36,01F0339S,2016-12-18 10:16:15,01H0447S,19.400,Y,2016-12-18 10:09:36+01F0339S; 2016-12-18 10:16...
9,32,2016-12-18 10:36:58,01F3696N,2016-12-18 10:43:35,01F3640N,8.000,Y,2016-12-18 10:36:58+01F3696N; 2016-12-18 10:38...


In [11]:
# 先檢查一下有沒有異常的資料
data[data.TripEnd == 'N'].shape

(0, 8)

In [12]:
# 先去除異常資料
data = data[data.TripEnd == 'Y']

In [13]:
# 然後乾脆刪掉 TripEnd 這欄
del data['TripEnd']
# 也可以用 data.drop('TripEnd', axis=1, inplace=True)

### 查看一下內容

In [14]:
# 前 5 筆
# 或 data.iloc[:5]
data.head(5)

,VehicleType,DetectionTime_O,GantryID_O,DetectionTime_D,GantryID_D,TripLength,TripInformation
0,31,2016-12-18 10:00:50,01F1045N,2016-12-18 10:29:01,01F0584N,53.4,2016-12-18 10:00:50+01F1045N; 2016-12-18 10:04...
1,31,2016-12-18 10:34:24,01F3525S,2016-12-18 10:46:09,01F3686S,20.2,2016-12-18 10:34:24+01F3525S; 2016-12-18 10:37...
2,31,2016-12-18 10:57:33,03F3854N,2016-12-18 11:15:23,01F3686S,33.6,2016-12-18 10:57:33+03F3854N; 2016-12-18 11:12...
3,31,2016-12-18 10:05:11,03F0525S,2016-12-18 10:07:39,03F0559S,12.1,2016-12-18 10:05:11+03F0525S; 2016-12-18 10:07...
4,31,2016-12-18 10:35:46,01F3185S,2016-12-18 11:05:08,01F3686S,54.1,2016-12-18 10:35:46+01F3185S; 2016-12-18 10:38...


In [15]:
# 第  12 筆
data.iloc[11]

VehicleType                                                       32
DetectionTime_O                                  2016-12-18 10:18:38
GantryID_O                                                  01F3640N
DetectionTime_D                                  2016-12-18 10:44:03
GantryID_D                                                  01F3525N
TripLength                                                        18
TripInformation    2016-12-18 10:18:38+01F3640N; 2016-12-18 10:21...
Name: 11, dtype: object

In [20]:
# 對我們來說，其實重要的只有 TripInformation 和 VehicleType
# 先只注意這兩項
data = data[['VehicleType', "TripInformation"]]
data.head(5)

,VehicleType,TripInformation
0,31,2016-12-18 10:00:50+01F1045N; 2016-12-18 10:04...
1,31,2016-12-18 10:34:24+01F3525S; 2016-12-18 10:37...
2,31,2016-12-18 10:57:33+03F3854N; 2016-12-18 11:12...
3,31,2016-12-18 10:05:11+03F0525S; 2016-12-18 10:07...
4,31,2016-12-18 10:35:46+01F3185S; 2016-12-18 10:38...


In [23]:
# 查看看小貨車資料
data.query('VehicleType==32')

,VehicleType,TripInformation
6,32,2016-12-18 10:00:21+05F0287N; 2016-12-18 12:27...
9,32,2016-12-18 10:36:58+01F3696N; 2016-12-18 10:38...
11,32,2016-12-18 10:18:38+01F3640N; 2016-12-18 10:21...
16,32,2016-12-18 10:32:41+01F0099N
27,32,2016-12-18 10:32:47+01F3185N
35,32,2016-12-18 10:47:00+01F3286N
41,32,2016-12-18 10:33:12+03F0337S; 2016-12-18 10:37...
51,32,2016-12-18 10:19:00+03A0041N
73,32,2016-12-18 10:08:37+05F0528N; 2016-12-18 10:15...
82,32,2016-12-18 10:21:55+01F0467N


In [25]:
# 或者查看看小客車資料
data[data.VehicleType==31]

,VehicleType,TripInformation
0,31,2016-12-18 10:00:50+01F1045N; 2016-12-18 10:04...
1,31,2016-12-18 10:34:24+01F3525S; 2016-12-18 10:37...
2,31,2016-12-18 10:57:33+03F3854N; 2016-12-18 11:12...
3,31,2016-12-18 10:05:11+03F0525S; 2016-12-18 10:07...
4,31,2016-12-18 10:35:46+01F3185S; 2016-12-18 10:38...
5,31,2016-12-18 10:00:35+03F3496N; 2016-12-18 10:03...
7,31,2016-12-18 10:50:12+01H0271N; 2016-12-18 10:54...
8,31,2016-12-18 10:09:36+01F0339S; 2016-12-18 10:16...
10,31,2016-12-18 10:59:07+03F2985N
12,31,2016-12-18 10:19:12+01F3460N
